## The Data Incubator Challenge #2

Consider $n$ buildings on a grid at poisitions $0$ to $n+1$. Each building has a height $h_i$. You can arrange the building in any order but you must the leave the slot at position $0$ open. Imagine a laser is shot just below the roof and to the left of a building. The laser travels any number of grid points until it encounters a building of a same height or taller or reaches the end of the grid, position $0$. For  example, consider $4$ buildings of height $3,3,4,$ and $1$ arranged at the grid points $1,2,3,$ and $4,$ respectively. The laser travels $1,1,3,$ and $1$ grid points for each of the buildings, respectively. Let's call the sum of the lasers' distances $V$. For this example, $V=6$. 

- Consider $4$ buildings of height $1,1,3,$ and $4$. For all possible configurations, what is the mean of sum of distances $V$?
- Consider $4$ buildings of height $1,1,3,$ and $4$. For all possible configurations, what is the standard deviation of the sum of distances $V$?
- Consider $10$ buildings of height $1,2,3,\ldots,10$. For all possible configurations, what is the mean of sum of distances $V$?
- Consider $10$ buildings of height $1,2,3,\ldots,10$. For all possible configurations, what is the standard deviation of the sum of distances $V$?
- Consider $20$ buildings of height $1,2,3,\ldots,20$. For all possible configurations, what is the mean of sum of distances $V$? The calculation of the standard deviation is not needed?

For all your questions, give your answer to $10$ places after the decimal point.

### Basic libraries

In [1]:
from itertools import permutations
import statistics
from math import factorial
import functools
import numpy as np

#### A function that computes $V$ for a given configuration.

In [2]:
def V(configuration):
    
    """Given a building configuration this function computes the distance laser travels 
    from each of the buildings in the given configuration and returns the sum of lasers'
     distances V for that configuration."""
    
    distances = []
    
    for i in range(1,len(configuration)):
        v = 0
        for j in range(1,i+1):            
            if configuration[i] == 0:
                v=0
                break
            if configuration[i] == configuration[i-j] and j == 1:
                v = 1
                break            
            if configuration[i] < configuration[i-j] and j==1:
                v = 1
                break
            if configuration[i] < configuration[i-j] and j > 1:
                v = j
                break
            if configuration[i] == configuration[i-j] and j > 1:
                v = j
                break
            if configuration[i] > configuration[i-j]:
                v = v+1
        distances.append(v)
    return sum(distances)


#### Computing the mean and the standard deviation

My first attempt of trying to calculate the mean and std values consisted of
 1. Apply $V$ to the permutations of the list of heights
 2. Append the values obtained in step 1 to a list
 3. Apply `statistics.mean` and `statistics.stdev` to the list obtained in step 2.

Although this approach works, it becomes extremely inefficient as the number of buildings increase. In fact I got a memory error after letting it run for a while for $n=13$ buildings. To remedy I tried using an array based approach which actually performed worse than the initial approach. Regardless, both approaches work for calculating the mean and std values for the first four questions, but both fail for the last question due to memory errors. To avoid running into memory errors I defined a function that computes $V$ for each permutation while keeping a cumulative sum of all $V$ values, and returns the mean by dividing the final sum by the number of different configurations. This new function performs slightly better then the previous two, and running it technically shouldn't lead to memory errors since, as oppose to the previous two functions, it doesn't store any lists or arrays in memory. However, this function will still fail to compute the desired answer in a reasonable time frame.


Below are the functions using each approach and comparison of their execution times.   

In [23]:
def V_sum_list(heights):
    
    V_values = []
    y = [0]   
    for x in permutations(y+ heights): #We add 0 to account for a slot that remains open when arranging n buildings at positions 1 to n+1.
        configuration = y+[i for i in x] #We add zero since the spot at zero position must remain open.
        V_values.append(V(configuration))
    return V_values
        

In [27]:
def V_sum_arr(heights):
    
    configurations = np.hstack((np.zeros((factorial(len(heights)+1),1)),
                                np.array(list(permutations([0]+heights))))) #Create a 2d array containing all configurations with open slot at position 0. 
    V_values = np.apply_along_axis(V, 1, configurations) # Apply V to configurations array along the columns to get a 1d array containing V values for each configuration.
    return V_values

In [28]:
def V_sum_mean(heights): 
    
    sum_of_V_values = 0
    for x in permutations([0]+heights): #We add 0 to account for a slot that remains open when arranging n buildings at positions 1 to n+1.
        sum_of_V_values += V([0]+list(x)) #We add zero since the spot at zero position must remain open.
    return sum_of_V_values/factorial(len(heights)+1)

In [29]:
%timeit V_sum_arr([1,1,3,4]).mean()

1.66 ms ± 35.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%timeit statistics.mean(V_sum_list([1,1,3,4]))

877 µs ± 56.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [31]:
%timeit V_sum_mean([1,1,3,4])

740 µs ± 9.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
%timeit V_sum_arr([1,1,3,4]).std()

1.77 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
%timeit statistics.stdev(V_sum_list([1,1,3,4]))

1.18 ms ± 8.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Answers to the questions

In [35]:
statistics.mean(V_sum_list([1,1,3,4]))

7.4

In [36]:
statistics.stdev(V_sum_list([1,1,3,4]))

1.8075472029123616

In [6]:
V_sum_mean([1,2,3,4,5,6,7,8,9,10])

24.23852813852814

In [37]:
statistics.stdev(V_sum_list([1,2,3,4,5,6,7,8,9,10]))

5.040321077600773